In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import matplotlib.pyplot  as plt
from PIL import Image
from pathlib import Path
import numpy as np

In [2]:
Data_dir = '../input/chest-xray/chest_xray'
Categories = ["Affected" , "Normal"]
for i in Categories:
    path = os.path.join(Data_dir,i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../input/chest-xray/chest_xray\\Affected'

In [5]:
Img_size = 224
new_array = cv2.resize(img_array, (Img_size,Img_size))

In [6]:
training_data = []
def create_training_data():
    for i in Categories:
        path = os.path.join(Data_dir,i)
        class_num = Categories.index(i) ## 0 for affected 1 for Normal
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (Img_size,Img_size))
            training_data.append([new_array, class_num])
create_training_data()

In [7]:
import random
random.shuffle(training_data)

In [8]:
x = []
y = []

In [9]:
for features,label in training_data:
    x.append(features)
    y.append(label)
x =np.array(x).reshape(-1,Img_size,Img_size, 1)

In [10]:
import pickle

pickle_out = open("x.pickle","wb")
pickle.dump(x, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [11]:
pickle_in = open("x.pickle", "rb")
x = pickle.load(pickle_in)

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

name = 'pneumonia-cnn-64x2-{}'.format(int(time.time()))

tensorboard = TensorBoard(log_dir='logs/{}'.format(name))

x = pickle.load(open("x.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

x = x/255.0

model = Sequential()
model.add(Conv2D(64,(3,3),input_shape=x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
tensorboard --logdir
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
x= np.array(x)
y= np.array(y)
model.fit(x, y, batch_size = 32,epochs=10,validation_split=0.3,callbacks=[tensorboard])


2022-11-19 14:31:44.893313: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-11-19 14:31:44.893361: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-11-19 14:31:44.999986: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 2 GPUs
2022-11-19 14:31:45.045023: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-11-19 14:31:45.045230: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-11-19 14:31:46.455089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 14:31:46.456271: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Epoch 1/10


2022-11-19 14:31:55.664579: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


  1/129 [..............................] - ETA: 21:30 - loss: 0.6825 - accuracy: 0.6562

2022-11-19 14:32:03.866707: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-11-19 14:32:03.866750: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/129 [..............................] - ETA: 2:22 - loss: 2.2244 - accuracy: 0.6875 

2022-11-19 14:32:04.978513: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-19 14:32:04.979150: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-11-19 14:32:05.143760: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 124 callback api events and 121 activity events. 
2022-11-19 14:32:05.156047: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-11-19 14:32:05.174044: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05

2022-11-19 14:32:05.178969: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05/b48db48982f6.trace.json.gz


  5/129 [>.............................] - ETA: 47s - loss: 1.2903 - accuracy: 0.6812 

2022-11-19 14:32:05.200838: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05

2022-11-19 14:32:05.202533: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05/b48db48982f6.memory_profile.json.gz
2022-11-19 14:32:05.203201: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05
Dumped tool data for xplane.pb to logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05/b48db48982f6.xplane.pb
Dumped tool data for overview_page.pb to logs/pneumonia-cnn-64x2-1668868304/train/plugins/profile/2022_11_19_14_32_05/b48db48982f6.overview_page.pb
Dumped tool data for input_pipeline.pb to logs/pneumonia-cnn-64x2-1668868304/train/plugins/

129/129 [==============================] - 23s 100ms/step - loss: 0.2915 - accuracy: 0.8880 - val_loss: 0.1879 - val_accuracy: 0.9192
Epoch 2/10
129/129 [==============================] - 10s 80ms/step - loss: 0.1612 - accuracy: 0.9402 - val_loss: 0.1421 - val_accuracy: 0.9471
Epoch 3/10
129/129 [==============================] - 9s 70ms/step - loss: 0.1193 - accuracy: 0.9571 - val_loss: 0.1433 - val_accuracy: 0.9493
Epoch 4/10
129/129 [==============================] - 10s 80ms/step - loss: 0.0825 - accuracy: 0.9693 - val_loss: 0.1456 - val_accuracy: 0.9516
Epoch 5/10
129/129 [==============================] - 9s 71ms/step - loss: 0.0636 - accuracy: 0.9783 - val_loss: 0.1762 - val_accuracy: 0.9334
Epoch 6/10
129/129 [==============================] - 10s 81ms/step - loss: 0.0384 - accuracy: 0.9861 - val_loss: 0.2765 - val_accuracy: 0.9186
Epoch 7/10
129/129 [==============================] - 10s 81ms/step - loss: 0.0389 - accuracy: 0.9863 - val_loss: 0.1982 - val_accuracy: 0.9454
Epoc